In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [ ]:
import P7_DataBreed
if True :
    oP7_DataBreed = P7_DataBreed.update_object_save(oP7_DataBreed, is_saved=False,is_new_attribute=True)
    oP7_DataBreed.show()


#### Object containing data breed model is loaded from dumped file.

In [ ]:
import p5_util

import P7_DataBreed
oP7_DataBreed = p5_util.object_load('./data/oP7_DataBreed.dump')
oP7_DataBreed.show()
oP7_DataBreed.breed_show()


# <font color='blus'>1. Breeds classification using pre-trained VGG16 model</font>
`VGG16` is a CNN network provided by / implemented in `Keras` with 16 layers.
<br>

While convolutional layers are used for features extraction, fully connected layers are used for classification.
<br>

Those 2 set of layers are used here to classify images as breeds, since breeds have already been learned by network.



In [ ]:
from keras.applications.vgg16 import VGG16
import p5_util

filename = './data/vgg16_pretrained.dump'

vgg16_pretrained = VGG16() 


if False :
    p5_util.object_dump(vgg16_pretrained,filename)
else : 
    vgg16_pretrained = p5_util.object_load(filename)
    

In [ ]:
type(vgg16_pretrained)

#### A breed and an image is selected and displayed

In [ ]:
oP7_DataBreed.breed_show()

List of images files names from selected breed is displayed.

In [ ]:
import P7_DataBreed

oP7_DataBreed.show_image_name('Mexican_hairless')

A selected PIL image is displayed

In [ ]:
import cv2

import p3_util
import p3_util_plot


import P7_DataBreed
import p7_util

breedname='Doberman'
imagename='n02107142_16400.jpg'
imagename='n02107142_4314.jpg'
dirbreed = 'n02107142-Doberman'

oP7_DataBreed_single = P7_DataBreed.P7_DataBreed()
oP7_DataBreed_single._dict_breed_sample=oP7_DataBreed._dict_breed_sample.copy()

list_restricted_image = [(breedname,[imagename])]
oP7_DataBreed_single.list_restricted_image = list_restricted_image

oP7_DataBreed_single.load(dirbreed=dirbreed, imagename=imagename)

p7_util.p7_image_pil_show(oP7_DataBreed_single._dict_img_pil,std_image_size=(500,375), is_title=False)

Selected image is pre-processed in order to feed pre-trained VGG16 network.

Prediction returns `y_pred`, a 1000 sized vector.
<br>

Built-in function `decode_predictions` allows to provide human readable results.

In [ ]:
from keras.preprocessing.image import load_img, img_to_array
from keras.applications import vgg16
import p3_util_plot

image_pathname = oP7_DataBreed._build_pathname(dirbreed,imagename)

# Charger l'image PIL
pil_image = load_img(image_pathname, target_size=(224, 224))  

# Convertir en tableau numpy
arr_image = img_to_array(pil_image)  

# Créer la collection d'images (un seul échantillon)
sample_count=1
arr_image = arr_image.reshape((sample_count, arr_image.shape[0], arr_image.shape[1], arr_image.shape[2]))  

# Prétraiter l'image comme le veut VGG-16
X_test = vgg16.preprocess_input(arr_image)  

# Predict image breed thanks to VGG16 pre-trained model
y_pred = vgg16_pretrained.predict(X_test)

#### Top prediction results are displayed.


In [ ]:
# Afficher les 3 classes les plus probables
top_results = 3
print("***Breed predictions top "+str(top_results)+" results")
[p3_util_plot.printmd(my_tuple) for my_tuple in vgg16.decode_predictions(y_pred, top=top_results)[0]]

# <font color='blus'> 2. Transfer Learning using `VGG16` pre-trained model</font>

Pre-trained model is used as a feature extractor.

It as learned discriminative features.

We consider we've a few sample to be trained  (3 breeds and close to 120 images) and we want to benefit of features extraction from already trained model.

<br>
Transfer learning seems to be a proper strategy to achieve classification over a few number of samples.


#### Download a VGG16 model that has been pre-trained with imageNet  : 
* last fully connected layers are removed; in the resulting model only convolutional layers stay.
* Image reshape is enforced to (224,224,3)

In [ ]:
from keras.applications import VGG16
 
vgg16_pretrained_conv = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))


In [ ]:
vgg16_pretrained_conv.summary()

In [ ]:
import numpy as np
from keras.preprocessing import image
batch_train_size = 36

train_dir = './data/ImagesPretrained/train'
 
datagen = image.ImageDataGenerator(rescale=1./255)

#-----------------------------------------------------------------------------
# train_generator is capable reading images from a given directory.
#-----------------------------------------------------------------------------
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch_train_size,
    class_mode='categorical',
    shuffle=True)


In [ ]:
nTrain = 360
nClass = 3
X_train = np.zeros(shape=(nTrain, 7, 7, 512))
y_train = np.zeros(shape=(nTrain,nClass))

#### Features are extracted feeding convolution layers with images issued from train directory.

* `y_train` holds encoded classes.
* `X_train` holds images used for training model.

In [ ]:
batch_id = 0
for inputs_batch, labels_batch in train_generator:
    
    print("Batch ID= "+str(batch_id))
    features_batch = vgg16_pretrained_conv.predict(inputs_batch)
    X_train[batch_id * batch_train_size : (batch_id + 1) * batch_train_size] = features_batch
    y_train[batch_id * batch_train_size : (batch_id + 1) * batch_train_size] = labels_batch
    batch_id += 1
    if batch_id * batch_train_size >= nTrain:
        break
         
X_train = np.reshape(X_train, (nTrain, 7 * 7 * 512))

In [ ]:
X_train.shape

#### Test dataset is loaded from tets directory

In [ ]:
import numpy as np
from keras.preprocessing import image
batch_test_size = 20

test_dir  = './data/ImagesPretrained/test'
 
datagen = image.ImageDataGenerator(rescale=1./255)

#-----------------------------------------------------------------------------
# test_generator is capable reading images from a given directory.
#-----------------------------------------------------------------------------
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=batch_test_size,
    class_mode='categorical',
    shuffle=False)


#### Features from Test set are extracted

In [ ]:
nTest = 101

X_test = np.zeros(shape=(nTest, 7, 7, 512))
y_test = np.zeros(shape=(nTest,nClass))
X_test.shape, y_test.shape

In [ ]:
batch_id = 0
for inputs_batch, labels_batch in test_generator:
    start = batch_id * batch_test_size
    end   = start + batch_test_size
    remain = X_test.shape[0] - start
    
    #end = min(end, remain)
    print("Batch ID= "+str(batch_id)+" : start= "+str(start)+ " end= "+str(end)+" remain= "+str(remain))
    features_batch = vgg16_pretrained_conv.predict(inputs_batch)

    if remain < batch_test_size:
        end  = start+remain
        X_test[start : end] = features_batch[:remain]
        y_test[start : end] = labels_batch[:remain]
    else :
        X_test[start : end] = features_batch
        y_test[start : end] = labels_batch
    batch_id += 1
    if batch_id * batch_test_size >= nTest:
        break

X_test = np.reshape(X_test, (nTest, 7 * 7 * 512))

In [ ]:
X_test.shape, y_test.shape

#### 2 full connected layers are created in order to complete convolutional layers.

3 classes take place in the model.

In [ ]:
from keras import models
from keras import layers
from keras import optimizers
 
model = models.Sequential()
# Fist dense layer is compliant with output of last convolutional layer from pre-trained model for transfer learning
model.add(layers.Dense(256, activation='relu', input_dim=7 * 7 * 512))
model.add(layers.Dropout(0.5))

# Last dense layer will classify amont nClass classes.
model.add(layers.Dense(nClass, activation='softmax'))

In [ ]:
model.summary()

#### Model is trained with previous features extracted from convolutional layers.

In [ ]:
model.compile(optimizer=optimizers.RMSprop(lr=1e-3),
              loss='categorical_crossentropy',
              metrics=['acc'])
 
history = model.fit(X_train,
                    y_train,
                    epochs=50,
                    batch_size=batch_size,
                    validation_data=(X_test,y_test))


In [ ]:
import matplotlib.pyplot as plt

#Plot the Loss Curves
plt.figure(figsize=[8,6])
plt.plot(history.history['loss'],'r',linewidth=1.0)
plt.plot(history.history['val_loss'],'b',linewidth=1.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves ',fontsize=16)
 
#Plot the Accuracy Curves
plt.figure(figsize=[8,6])
plt.plot(history.history['acc'],'r',linewidth=1.0)
plt.plot(history.history['val_acc'],'b',linewidth=1.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16)

#### Wrongly classified breeds

In [ ]:
fnames = test_generator.filenames
 
ground_truth = test_generator.classes
 
label2index = test_generator.class_indices
 
# Getting the mapping from class index to class label
idx2label = dict((v,k) for k,v in label2index.items())
 
predictions = model.predict_classes(X_test)
prob = model.predict(X_test)
 
errors = np.where(predictions != ground_truth)[0]
print("Number of errors = {}/{}".format(len(errors),nTest))

In [ ]:
from keras.preprocessing.image import load_img

for i in range(len(errors)):
    pred_class = np.argmax(prob[errors[i]])
    pred_label = idx2label[pred_class]
     
    print('Original label:{}, Prediction :{}, confidence : {:.3f}'.format(
        fnames[errors[i]].split('/')[0],
        pred_label,
        prob[errors[i]][pred_class]))
     
    original = load_img('{}/{}'.format(test_dir,fnames[errors[i]]))
    plt.imshow(original)
    plt.show()

In [ ]:
150/600

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.layers import Dense
import p5_util
if False :
    # Load ImageNet pre-trained model from VGG-16 
    # Option include_top=False leads to remove last Fully connected layer.
    # VGG16 model is downloaed from network.
    #vgg16_pretrained_model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    vgg16_pretrained_model = VGG16(weights="imagenet", include_top=True, input_shape=(224, 224, 3))
    p5_util.object_dump(vgg16_pretrained_model,'./data/vgg16_pretrained_model.dump')
else :
    vgg16_pretrained_model = p5_util.object_load('./data/vgg16_pretrained_model.dump')
vgg16_pretrained_model.summary()

#### An image is selected among breeds

In [ ]:
oP7_DataBreed.breed_show()

In [ ]:
breedname='dingo'

In [ ]:
oP7_DataBreed.show_image_name(breedname, is_sample_show=False)

In [ ]:
from keras.applications.imagenet_utils import decode_predictions
from keras.applications import vgg16
import p7_util

imagename = 'n02115641_4970.jpg'
pil_image = oP7_DataBreed.read_image(breedname, imagename)

keras_image = p7_util.p7_pil_to_keras_image(pil_image, is_show=True)
# prepare the image for the VGG model
processed_image = vgg16.preprocess_input(keras_image.copy())
 
# get the predicted probabilities for each class
predictions = vgg16_pretrained_model.predict(processed_image)
# print predictions
 
# convert the probabilities to class labels
# We will get top 5 predictions which is the default
pred_breedname_result = decode_predictions(predictions)
pred_breedname = pred_breedname_result[0][0][1]
pred_proba = pred_breedname_result[0][0][2]*100
print("Original breed name = "+breedname+" / Greatest likehood breed name= "+str(pred_breedname)+" Prob={0:0.2F}".format(pred_proba))

#### Transfer learning model is built
* Output of all layers previous top layer are extracted.
* A new fully connected layer (dense layer) is built using previous layer as an input layer

In [ ]:
from keras import models
from keras import layers

# Outputs parameters are retrieved from previous layers
vgg16_model_tensor = vgg16_pretrained_model.output

# A new fully connected layer is then added in order to classify nbreeds breeds.
# Activation function is Softmax; This last layer is applied to the data input vgg16_model_parameters
vgg16_fc_tensor = layers.Dense(nbreed, activation='softmax')(vgg16_model_tensor)

# New VGG16 model for transfer learning is defined with : 
# * All layers except last layer are issued from 
# * fully connected layer previously defined.
model_vgg16_transfer_learning = models.Model(inputs=vgg16_pretrained_model.input, outputs=vgg16_fc_tensor)
type(model_vgg16_transfer_learning)

#### Compilation : shape of all internal layers are computed by compiler

In [ ]:
# Compiler le modèle 
from keras import optimizers

model_vgg16_transfer_learning.compile(loss="categorical_crossentropy", optimizer=optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

In [ ]:
import numpy as np
import p5_util

filename='./data/arr_keras_X_y_train_test.dump'
(X_train,X_test, y_train, y_test) = p5_util.object_load(filename)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)
if False :
    n_xtrain = int((X_train.shape[0]*X_train.shape[1]*X_train.shape[2]*X_train.shape[3])/(7*7*512))
    print(n_xtrain)

    n_ytrain = int((y_train.shape[0]*y_train.shape[1])/(7*7*512))
    print(n_ytrain)

    X_train = X_train.reshape((n_xtrain, 7, 7, 512))
    #y_train = y_train.reshape((n_ytrain, 7, 7, 512))
    print(X_train.shape, y_train.shape)
    np.unique(y_train)

In [ ]:
n_ytrain = int((y_train.shape[0]*y_train.shape[1])/(7*7*3))
print(n_ytrain)
y_train.shape,7*7*3

In [ ]:
#help(model_vgg16_transfer_learning.fit)
model_vgg16_transfer_learning.summary()

In [ ]:
y_train.shape

In [ ]:
# Entraîner sur les données d'entraînement (X_train, y_train)
model_info = model_vgg16_transfer_learning.fit(X_train, y_train, epochs=1, batch_size=1, verbose=2)

## Total fine-tuning

On remplace la dernière couche fully-connected du réseau pré-entraîné par un classifieur adapté au nouveau problème (SVM, régression logistique...) et initialisé de manière aléatoire. Toutes les couches sont ensuite entraînées sur les nouvelles images. 

La stratégie #1 doit être utilisée lorsque la nouvelle collection d'images est grande : dans ce cas, on peut se permettre d'entraîner tout le réseau sans courir le risque d'overfitting. De plus, comme les paramètres de toutes les couches (sauf de la dernière) sont initialement ceux du réseau pré-entraîné, la phase d'apprentissage sera faite plus rapidement que si l'initialisation avait été aléatoire.